## Exploring experiments

In [9]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

from os import path
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from trojan_defender import datasets
from pymongo import MongoClient

In [3]:
# config logging
logging.basicConfig(level=logging.ERROR)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

dump_folder = '/home/Edu/saliency'

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [4]:
clean = datasets.mnist()

def detect(directory):
    model, dataset, metadata = experiment.load(directory)
    score = saliency.score(model, clean)
    return score

In [16]:
def evaluate_experiment(experiment, dump=False):
    # directory = model['directory']
    experiment = list(experiment)
    mean_acc = np.mean([model['metrics']['accuracy_score'] for model in experiment])
    scores = [detect(model['directory']) for model in experiment]
    return mean_acc, scores

In [6]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [33]:
# noisy models
noisy = con.find({'dataset.name': 'MNIST',
                  'group_name': 'big_mnist_exp_12pm_2',
                  'dataset.dynamic_mask': True})

# baseline
baseline = con.find({'dataset.name': 'MNIST',
                     'group_name': 'big_mnist_exp_12pm_2',
                     'metrics.patch_success_rate': {"$exists": False}})

# attacks
# 'metrics.patch_success_rate': {"$lt": 0.05},
attacked = con.find({'dataset.name': 'MNIST',
                     'group_name': 'mnist_5pm2',
                     'dataset.dynamic_mask': False,
                     'dataset.dynamic_pattern': False,
                     'dataset.fraction': {'$gt': 0.09},
                     'dataset.proportion': 0.01})

In [34]:
attacked = list(attacked)

In [35]:
len(attacked)

8

In [36]:
res = evaluate_experiment(attacked)
scores = [r[1] for r in res]
all([s > 0.5 for s in scores])

IndexError: invalid index to scalar variable.

In [ ]:
res_baseline = [detection_round(model) for model in baseline]

In [ ]:
res_noisy = [detection_round(model) for model in noisy]

## Plots

In [ ]:
success = [r[1] for r in res]
mode_changes = [r[3] for r in res]

plt.scatter(success, mode_changes, alpha=0.5)